In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import BaggingRegressor
import lzma
import cloudpickle
import os

In [2]:
df = pd.read_pickle('/tmp/all.pkl')
display(df.nunique())
display(df.isnull().sum())

type                    5
zone_type               4
city_code            1893
prefecture             47
city                 1891
city2               70198
nearest_sta          8614
nearest_sta_dist       34
price                 745
tubo_price            588
layout                 66
area                  159
m2_price              551
shape                   9
frontage              500
floor_area            130
building_year          79
structure              35
purpose               289
future_purpose          6
front_road_dir          9
front_road_type        14
front_road_width      564
city_plan              16
building_cov            6
floor_cov              17
time                   69
renovation              2
notes                  36
dtype: int64

type                      0
zone_type           1411809
city_code                 0
prefecture                0
city                      0
city2                 10281
nearest_sta          708538
nearest_sta_dist     732460
price                     0
tubo_price          3184891
layout              4125012
area                      0
m2_price            3184891
shape               1422496
frontage            1725117
floor_area          3214801
building_year       2512836
structure           2474225
purpose             2533470
future_purpose      2990559
front_road_dir      1423781
front_road_type     1500558
front_road_width    1536727
city_plan            663637
building_cov         889683
floor_cov            889683
time                      0
renovation          4172789
notes               4607820
dtype: int64

In [3]:
df = pd.read_pickle('/tmp/all.pkl')

for col in df.columns:
    if not (df.dtypes[col] == 'float' or df.dtypes[col] == 'int'):
        df = df.drop(columns=col)
        
# df['y'] = np.log(df['price'])
df['y'] = np.log(df['price'] / df['area'])

for col in df.columns:
    if (df.dtypes[col] == 'float' or df.dtypes[col] == 'int') and col != 'y':
        df2 = df[[col, 'y']].dropna()
        df2['col_log'] = np.log1p(df2[col])
        print(col)
        print(pearsonr(df2[col], df2['y'])[0])
        print(pearsonr(df2['col_log'], df2['y'])[0])
        
display(df.corr())

city_code
-0.11772531361956574
0.04139372722727577
nearest_sta_dist
-0.4744470349182286
-0.492375255551253
price
0.1413217619175039
0.8642047024362377
tubo_price
0.5709154320466656
0.9913958191756616
area
-0.5729643265611083
-0.6837566794150942
m2_price
0.5706320333096238
0.9913965078885248
frontage
-0.39148899138301074
-0.4106919607315005
floor_area
0.059657581518301216
0.034530561818274394
building_year
0.4514290413158638
0.451557400713963
front_road_width
0.05630136109703938
0.0684388896171183
building_cov
0.13798717111730688
0.11483262757387637
floor_cov
0.26230576684149864
0.19182035737147957
time
-0.03549754629036588
-0.03554724811342919


,city_code,nearest_sta_dist,price,tubo_price,area,m2_price,frontage,floor_area,building_year,front_road_width,building_cov,floor_cov,time,y
city_code,1.000000,0.003306,-0.017349,-0.066873,0.022453,-0.066819,0.022656,0.020341,-0.069289,-0.066868,0.087277,0.029850,0.013341,-0.117725
nearest_sta_dist,0.003306,1.000000,-0.051712,-0.243605,0.241246,-0.243476,0.215319,-0.027813,-0.058558,-0.019942,-0.187318,-0.228697,-0.004823,-0.474447
price,-0.017349,-0.051712,1.000000,0.314606,0.026389,0.314550,0.099438,0.308418,0.026475,0.116600,0.036219,0.073691,-0.005140,0.141322
tubo_price,-0.066873,-0.243605,0.314606,1.000000,-0.160307,0.999779,-0.179867,NaN,NaN,0.068975,0.070828,0.200501,-0.030898,0.570915
area,0.022453,0.241246,0.026389,-0.160307,1.000000,-0.160235,0.743140,0.608349,-0.157159,0.155271,-0.057406,-0.102151,0.020245,-0.572964
m2_price,-0.066819,-0.243476,0.314550,0.999779,-0.160235,1.000000,-0.179771,NaN,NaN,0.068984,0.070832,0.200450,-0.030894,0.570632
frontage,0.022656,0.215319,0.099438,-0.179867,0.743140,-0.179771,1.000000,0.476604,-0.220741,0.146631,0.020561,0.002263,0.014254,-0.391489
floor_area,0.020341,-0.027813,0.308418,NaN,0.608349,NaN,0.476604,1.000000,-0.125432,0.314077,0.211962,0.277026,-0.009989,0.059658
building_year,-0.069289,-0.058558,0.026475,NaN,-0.157159,NaN,-0.220741,-0.125432,1.000000,-0.066946,-0.080590,-0.051043,0.134744,0.451429
front_road_width,-0.066868,-0.019942,0.116600,0.068975,0.155271,0.068984,0.146631,0.314077,-0.066946,1.000000,0.183241,0.226906,0.002237,0.056301


In [4]:
print('predictability of zone_type and city_plan and city_code')

df = pd.read_pickle('/tmp/all.pkl')

df = df.sort_values('time', kind='stable')

group = ['prefecture', 'city']
# group = ['prefecture', 'city', 'city2']

df['time_prev'] = df.groupby(group)['time'].shift(1)

df['city_plan_pred'] = df.groupby(group)['city_plan'].shift(1)
df.loc[df['time_prev'] == df['time'], 'city_plan_pred'] = np.nan
df['city_plan_pred'] = df.groupby(group)['city_plan_pred'].transform(lambda x: x.ffill())

df['city_code_pred'] = df.groupby(group)['city_code'].shift(1)
df.loc[df['time_prev'] == df['time'], 'city_code_pred'] = np.nan
df['city_code_pred'] = df.groupby(group)['city_code_pred'].transform(lambda x: x.ffill())

display(df[['city_plan', 'city_plan_pred']].dropna())

print(np.mean(df['city_plan_pred'] != df['city_plan']))
print(np.mean(df['city_code_pred'] != df['city_code']))

c = df.groupby(group)['city_code'].apply(lambda x: x.unique().shape[0])
print(c)
print(c[c != 1])
# print(df.groupby(['prefecture', 'city'])['city_code'].nth(0).to_dict())

predictability of zone_type and city_plan and city_code


,city_plan,city_plan_pred
11408,第２種中高層住居専用地域,第２種中高層住居専用地域
11424,準工業地域,都市計画区域外
11425,準工業地域,都市計画区域外
11426,第１種住居地域,都市計画区域外
11441,市街化調整区域,第２種中高層住居専用地域
...,...,...
4880199,準住居地域,市街化調整区域
4880200,都市計画区域外,市街化調整区域
4880201,市街化調整区域,市街化調整区域
4880202,第１種住居地域,市街化調整区域


0.785862684047084
0.004259862034484207
prefecture  city   
三重県         いなべ市       1
            三重郡川越町     1
            三重郡朝日町     1
            三重郡菰野町     1
            亀山市        1
                      ..
鹿児島県        霧島市        1
            鹿児島市       1
            鹿児島郡三島村    1
            鹿児島郡十島村    1
            鹿屋市        1
Name: city_code, Length: 1893, dtype: int64
Series([], Name: city_code, dtype: int64)


In [5]:
# available features
features = [
    # 'zone_type',
    'city_plan',
    'city_code',
    'prefecture',
    'city',
    'city2',
    'nearest_sta',
    'nearest_sta_dist',
    'age',
    # 'structure',
    'area',
    'floor_area',
    # 'layout', # many nan
    # 'front_road_dir',
    # 'front_road_type',
    'front_road_width',
    # 'building_cov',
    # 'floor_cov',
    'building_year',
    'time',
    # 'choume'
    # 'sawa',
    # 'numa',
    # 'tani',
    # 'ken',
    'city_code_low',
    # 'city_code_low2',
]

In [6]:
def show_importance(model, features):
    importances = model.feature_importances_
    feature_imp = pd.DataFrame(zip(importances, features), columns=['value', 'feature'])
    feature_imp = feature_imp.sort_values('value')
    display(feature_imp)

In [7]:
class RemlModel:
    def __init__(self, df_all):
        self._city_code_map = df_all.groupby(['prefecture', 'city'])[['city_code']].nth(0)
    
        self._unique_values = {}
        for col in df_all.columns:
            self._unique_values[col] = df_all[col].sort_values().unique().tolist()
        
        x = df_all['prefecture']
        x = x + ':' + df_all['city']
        x = x + ':' + df_all['city2']
        x = x + ':' + df_all['nearest_sta']
        self._unique_values['prefecture_city_city2_nearest_sta'] = x.sort_values().unique().tolist()
            
        self._dtypes = df_all.dtypes

    def fit(self, df):
        df = df.copy()
        
        cat_features = [
            'city_plan',
            'prefecture',
            'city',
            'city2',
            'nearest_sta',
            # 'structure',
            # 'layout',
            # 'front_road_dir',
            # 'front_road_type',
        ]
        num_features = [x for x in features if x not in cat_features]
        # num_features = [features.index(x) for x in num_features]
        # cat_features = [features.index(x) for x in cat_features]
        
        def create_model(alpha=None):
            model = lgb.LGBMRegressor(
                alpha=alpha,
                objective='regression' if alpha is None else 'quantile',
                colsample_bytree=0.5,
                # subsample=0.5,
                # subsample_freq=1,
                # extra_trees=True,
                random_state=1, 
                n_jobs=-1
            )
            model = BaggingRegressor(model, n_estimators=10, random_state=1)

            num_transformer = Pipeline([
                ("imputer", SimpleImputer(strategy="median")), 
                # ("scaler", StandardScaler()),
            ])

            cat_transformer = Pipeline([
                ("le", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
            ])

            model = Pipeline([
                ('ct', ColumnTransformer([
                    # ("num", 'passthrough', num_features),
                    ("num", num_transformer, num_features),
                    ("cat", cat_transformer, cat_features),
                ])),
                ('model', model),
            ])

            # model = BaggingRegressor(model, n_estimators=10, random_state=1)
            
            return model
        
        df = df.sample(frac=1, random_state=1) # for ordinal encoder
        
        # y = np.log(df['price'])
        y = np.log(df['price'] / df['area'])
        # plt.hist(y, bins=100)
        # plt.show()
        X = self.calc_features(df)
        
        model = create_model()
        model.fit(X, y)
        
        show_importance(model.named_steps['model'].estimators_[0], num_features + cat_features)
        
        self._model = model
        
        model = create_model(alpha=0.1)
        model.fit(X, y)
        self._model_10 = model
        model = create_model(alpha=0.9)
        model.fit(X, y)
        self._model_90 = model
    
    def predict(self, df):
        df = df.copy()
        
        for col in df.columns:
            if col in self._dtypes:
                df[col] = df[col].astype(self._dtypes[col])
        
        X = self.calc_features(df)
        # return np.exp(self._model.predict(X))
        
        df_res = pd.DataFrame(index=df.index)
        df_res['price'] = np.exp(self._model.predict(X)) * df['area'].values
        df_res['price_10'] = np.exp(self._model_10.predict(X)) * df['area'].values
        df_res['price_90'] = np.exp(self._model_90.predict(X)) * df['area'].values
        
        return df_res
    
    def metadata(self):
        return {
            'unique_values': self._unique_values,
        }
    
    def calc_features(self, df):
        df_orig = df.reset_index()
        df = df_orig.copy()
        
        df = df.merge(self._city_code_map, how='inner', on=['prefecture', 'city'])
        df = df.set_index('index').loc[df_orig['index']]
        
#         from kanjize import kanji2number
#         def conv_kanji(x):
#             if isinstance(x, float):
#                 return 0
#             print(x)
#             return kanji2number(x)
        
#         df['choume'] = df['city2'].str.extract(r'([一二三四五六七八九十]+)丁目')
#         df['choume'] = df['choume'].apply(conv_kanji).astype('int')
        
        # df['sawa'] = df['city2'].str.contains('沢')
        # df['numa'] = df['city2'].str.contains('沼')
        # df['tani'] = df['city2'].str.contains('谷')

        # df['ken'] = df['prefecture'].str.contains('県')
        df['city_code_low'] = df['city_code'] % 1000
        # df['city_code_low2'] = df['city_code'] % 100
    
        df['age'] = df['time'] - df['building_year'] * 100
            
        # display(df)
        # display(df_orig)
            
        df = df[features]

        return df

In [9]:
df = pd.read_pickle('/tmp/all.pkl')
df = df.iloc[::100]

model = RemlModel(df)

df = df.drop(columns='city_code')

model.fit(df)
# print(model.metadata())

data = cloudpickle.dumps(model)
data = lzma.compress(data)
model_name = '20230220_baseline'
with open('/home/jovyan/data/{}.xz'.format(model_name), 'wb') as f:
    f.write(data)
print('model size {}'.format(os.path.getsize('/home/jovyan/data/{}.xz'.format(model_name))))

cv = KFold(shuffle=True, random_state=1)

def calc_score(y, y_pred):
    return r2_score(np.log(y), np.log(y_pred))
    # return pearsonr(np.log(y), np.log(y_pred))[0]

scores = []
for train_idx, val_idx in cv.split(df):
    model.fit(df.iloc[train_idx])
    df_res = model.predict(df.iloc[val_idx])
    y_pred = df_res['price']
    y = df['price'].iloc[val_idx]
    score = calc_score(y, y_pred)
    scores += [score]
    print('score {}'.format(score))

print(scores)
print(np.mean(scores))

,value,feature
6,97,building_year
1,139,nearest_sta_dist
4,145,floor_area
2,149,age
12,164,city2
5,170,front_road_width
13,184,nearest_sta
9,208,city_plan
7,227,time
10,229,prefecture


model size 3034888


,value,feature
6,109,building_year
1,133,nearest_sta_dist
4,145,floor_area
2,158,age
12,158,city2
5,159,front_road_width
13,195,nearest_sta
9,197,city_plan
10,208,prefecture
7,222,time


score 0.7636488691646421


,value,feature
6,115,building_year
4,133,floor_area
1,140,nearest_sta_dist
2,156,age
5,171,front_road_width
12,175,city2
13,197,nearest_sta
9,208,city_plan
10,218,prefecture
7,227,time


...

score 0.7575271321446635


,value,feature
6,108,building_year
1,123,nearest_sta_dist
2,142,age
5,144,front_road_width
12,158,city2
4,165,floor_area
13,191,nearest_sta
9,226,city_plan
7,236,time
11,242,city


score 0.7497001885392885


,value,feature
6,130,building_year
1,134,nearest_sta_dist
2,136,age
4,151,floor_area
5,165,front_road_width
12,179,city2
13,192,nearest_sta
9,194,city_plan
7,232,time
10,236,prefecture


score 0.7539894871132655
[0.7636488691646421, 0.7417300351630937, 0.7575271321446635, 0.7497001885392885, 0.7539894871132655]
0.7533191424249907
